In [ ]:
import cv2
import numpy as np

# Load the four images to be fused
img1 = cv2.imread("E:/S4/Project/Main Project/images/1.jpeg")
img2 = cv2.imread("E:/S4/Project/Main Project/images/2.jpeg")
img3 = cv2.imread("E:/S4/Project/Main Project/images/3.jpeg")
img4 = cv2.imread("E:/S4/Project/Main Project/images/4.jpeg")

# Resize the images to the same size
img1 = cv2.resize(img1, (680, 400),interpolation=cv2.INTER_AREA)
img2 = cv2.resize(img2, (680, 400),interpolation=cv2.INTER_AREA)
img3 = cv2.resize(img3, (680, 400),interpolation=cv2.INTER_AREA)
img4 = cv2.resize(img4, (680, 400),interpolation=cv2.INTER_AREA)

gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
gray3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
gray4 = cv2.cvtColor(img4, cv2.COLOR_BGR2GRAY)

fused1 = cv2.addWeighted(gray1, 0.5, gray2, 0.5, 0)
fused2 = cv2.addWeighted(gray3, 0.5, gray4, 0.5, 0)

final_fused = cv2.add(fused1, fused2)

cv2.imshow('final_fused.jpg', final_fused)
cv2.imwrite('E:/S4/Project/Main Project/images/output.jpg', final_fused)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
x = final_fused.shape[1] // 2 - 50
y = final_fused.shape[0] // 2 - 50
width = 100
height = 100
effective_area = final_fused[y:y+height, x:x+width]
N = 256
effective_area_resized = cv2.resize(effective_area, (N, N))
cv2.imwrite('E:/S4/Project/Main Project/images/effective_area.jpg', effective_area_resized)

True

In [12]:
import pywt
import numpy as np
import cv2

# Load the image
image = cv2.imread('E:/S4/Project/Main Project/images/effective_area.jpg', 0)

# Define the wavelet to use (in this case, we will use the Haar wavelet)
wavelet = pywt.Wavelet('haar')

# Decompose the image using the Lifting Wavelet Transform
coeffs = pywt.wavedec2(image, wavelet, mode='symmetric', level=1)

# Extract the approximation coefficients (which correspond to the low frequency information)
A = coeffs[0]

# Save the approximation coefficients as an image
cv2.imwrite('E:/S4/Project/Main Project/images/approximation_coefficients.jpg', A)


True

In [18]:
import numpy as np
import cv2

# Load the low-frequency coefficient image
A = cv2.imread('E:/S4/Project/Main Project/images/approximation_coefficients.jpg', 0)

# Define the block size
n = 8

# Divide the image into blocks of size n x n
blocks = [A[j:j+n, i:i+n] for (j, i) in np.ndindex(A.shape[0]//n, A.shape[1]//n)]

# Apply QR decomposition to each block
Q_blocks = []
R_blocks = []
for block in blocks:
    Q, R = np.linalg.qr(block)
    Q_blocks.append(Q)
    R_blocks.append(R)

# Save the Q and R blocks as separate images
for i in range(len(blocks)):
    cv2.imwrite(f'E:/S4/Project/Main Project/images/block/Q_block_{i}.jpg', Q_blocks[i])
    cv2.imwrite(f'E:/S4/Project/Main Project/images/block/R_block_{i}.jpg', R_blocks[i])


In [19]:
import numpy as np

# Define the size of the matrix
k = 4

# Generate a random matrix of size k x k
A = np.random.rand(k, k)

# Compute the 2-norm of each column of the matrix
col_norms = np.linalg.norm(A, ord=2, axis=0)

# Construct a diagonal matrix of the column norms
D = np.diag(col_norms)

# Compute the 2-norm of the original matrix using the singular value decomposition (SVD)
svd = np.linalg.svd(A)
matrix_norm = svd[1][0]

# Print the original matrix and the 2-norm matrix
print("Original matrix:")
print(A)
print("2-norm matrix:")
print(D)
print("2-norm of the original matrix:")
print(matrix_norm)

Original matrix:
[[0.89269835 0.32788304 0.00227125 0.36102844]
 [0.30858679 0.36697288 0.03186704 0.60585964]
 [0.84592991 0.65732737 0.87682027 0.40692858]
 [0.11051186 0.12717982 0.24582509 0.85385711]]
2-norm matrix:
[[1.27277116 0.         0.         0.        ]
 [0.         0.83092139 0.         0.        ]
 [0.         0.         0.91118847 0.        ]
 [0.         0.         0.         1.17986027]]
2-norm of the original matrix:
1.8862917830792398


In [21]:
import cv2
import numpy as np

# Normalize 2-norm values to range [0, 255]
cv2.normalize(matrix_norm, matrix_norm, 0, 255, cv2.NORM_MINMAX)

# Binarize 2-norm values
_, binarized_img = cv2.threshold(matrix_norm, 128, 255, cv2.THRESH_BINARY)

# Save feature image
cv2.imwrite('E:/S4/Project/Main Project/images/feature_image.jpg', binarized_img)


True